In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
# Load the data from the 'Loan_Data.csv' file into a DataFrame named df
df = pd.read_csv('Loan_Data.csv')
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [3]:
df.shape

(10000, 8)

In [4]:
df.dtypes

customer_id                   int64
credit_lines_outstanding      int64
loan_amt_outstanding        float64
total_debt_outstanding      float64
income                      float64
years_employed                int64
fico_score                    int64
default                       int64
dtype: object

Le DataFrame contient 10k observations et 8 variables, toutes numériques.

In [5]:
# Find columns with NaN values
# Count NaN values for each column
nan_counts = df.isna().sum()

# Filter and print only the columns with NaN values and their counts
nan_columns_counts = nan_counts[nan_counts > 0]
nan_columns_counts

Series([], dtype: int64)

La DataFrame ne contient aucun NaN.

In [9]:
def standard_columns(column):
    return (column - column.mean()) / column.std() 

# Standardize the floats variables
df[df.select_dtypes(["float"]).columns] = df[
    df.select_dtypes(["float"]).columns 
].apply(standard_columns)
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,0.747058,-0.724812,0.398535,5,605,0
1,7442532,5,-1.548297,-0.073963,-2.161768,2,572,1
2,2256073,0,-0.560481,-1.009645,-0.207909,4,602,0
3,4885975,0,0.427024,-0.938137,0.215073,5,612,0
4,4700614,1,-1.979634,-1.048728,-2.321198,6,631,0


In [10]:
col_target = "default"
col_features = df.columns.tolist()[1:-1] # Get all the columns except the customer_id and the target columns

X = np.array(df[col_features]).astype(float) # Convert the features to a numpy array
y = np.array(df[col_target]) # Convert the target to a numpy array

model = sm.Logit(y, X) # Create a logistic regression model
result = model.fit() # Fit the model

summary = result.summary() # Get the summary of the model
summary

Optimization terminated successfully.
         Current function value: 0.001995
         Iterations 19


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9994
Method:                           MLE   Df Model:                            5
Date:                Tue, 06 Aug 2024   Pseudo R-squ.:                  0.9958
Time:                        17:23:18   Log-Likelihood:                -19.952
converged:                       True   LL-Null:                       -4790.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            58.1552     13.362      4.352      0.000      31.966      84.344
x2             4.2037      1.274      3.299      0.001       1.707       6.701
x3            46.1399     10.738      4.297      0.000      25.095      67.185
x4           -31.9597      7.549     -4.234      0.000     -46.755     -17.165
x5           -22.4059      5.162     -4.341      0.000     -32.523     -12.289
x6            -0.2119      0.048     -4.377      0.000      -0.307      -0.117
==============================================================================

Possibly complete quasi-separation: A fraction 0.99 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""